In [1]:
from qectostim.codes.base.four_two_two import FourQubit422Code
from qectostim.codes.base.six_two_two import SixQubit622Code
from qectostim.codes.base.rotated_surface import RotatedSurfaceCode
from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise


In [16]:

code = FourQubit422Code()
noise = CircuitDepolarizingNoise(p1=1e-1, p2=1e-1)
exp = CSSMemoryExperiment(code=code, noise_model=noise, rounds=3, basis="Z")

ideal = exp.to_stim()
noisy = noise.apply(ideal)

print(ideal)

# dem = noisy.detector_error_model(decompose_errors=True)
# print("DEM summary:", dem)
res = exp.run_decode(1_000_000)
res_no_dec = exp.run_no_decode(1_000_000)

print("Results with decoding:", res)
print("Results without decoding:", res_no_dec)

QUBIT_COORDS(0, 0) 0
QUBIT_COORDS(1, 0) 1
QUBIT_COORDS(1, 1) 2
QUBIT_COORDS(0, 1) 3
QUBIT_COORDS(0.5, 0.5) 4
QUBIT_COORDS(0.5, 0.5) 5
R 0 1 2 3 4 5 6
TICK
H 4
TICK
H 4
CX 0 5 1 5 2 5 3 5 0 6 1 6 2 6 3 6
MR 4
DETECTOR(0.5, 0.5, 0) rec[-1]
MR 5 6
DETECTOR(0.5, 0.5, 0) rec[-2]
DETECTOR(0, 0, 0) rec[-1]
SHIFT_COORDS(0, 0, 1)
H 4
TICK
H 4
CX 0 5 1 5 2 5 3 5 0 6 1 6 2 6 3 6
MR 4
DETECTOR(0.5, 0.5, 0) rec[-4] rec[-1]
MR 5 6
DETECTOR(0.5, 0.5, 0) rec[-5] rec[-2]
DETECTOR(0, 0, 0) rec[-4] rec[-1]
SHIFT_COORDS(0, 0, 1)
H 4
TICK
H 4
CX 0 5 1 5 2 5 3 5 0 6 1 6 2 6 3 6
MR 5 6
DETECTOR(0.5, 0.5, 0) rec[-4] rec[-2]
DETECTOR(0, 0, 0) rec[-3] rec[-1]
SHIFT_COORDS(0, 0, 1)
M 0 1 2 3
DETECTOR(0.5, 0.5, 1) rec[-4] rec[-3] rec[-2] rec[-1] rec[-6]
DETECTOR(0, 0, 1) rec[-4] rec[-3] rec[-2] rec[-1] rec[-5]
OBSERVABLE_INCLUDE(0) rec[-4] rec[-3]
[run_decode] --- starting ---
[run_decode] shots           = 1000000
[run_decode] decoder_name    = None
[run_decode] code type       = <class 'qectostim.codes.base.fou

In [12]:
# compare_rotated_surface_code.py
from __future__ import annotations

import collections
import itertools
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import stim

from qectostim.codes.base.rotated_surface import RotatedSurfaceCode
from qectostim.experiments.memory import CSSMemoryExperiment


# ----------------------------
# 1) Build circuits
# ----------------------------

def build_ours(d: int, rounds: int, basis: str) -> stim.Circuit:
    code = RotatedSurfaceCode(d)
    exp = CSSMemoryExperiment(code=code, rounds=rounds, basis=basis, noise_model=None)
    return exp.to_stim()


def build_stim(d: int, rounds: int, basis: str) -> stim.Circuit:
    # This is Stim’s standard generator name.
    # Works on all recent stim versions:
    #   "surface_code:rotated_memory_x" or "surface_code:rotated_memory_z"
    name = f"surface_code:rotated_memory_{basis.lower()}"
    try:
        return stim.Circuit.generated(name, distance=d, rounds=rounds)
    except TypeError:
        # Some versions want "d" instead of "distance".
        return stim.Circuit.generated(name, d=d, rounds=rounds)


# ----------------------------
# 2) Strip noise + normalize ticks
# ----------------------------

NOISE_OPS = {
    "DEPOLARIZE1",
    "DEPOLARIZE2",
    "X_ERROR",
    "Y_ERROR",
    "Z_ERROR",
    "PAULI_CHANNEL_1",
    "PAULI_CHANNEL_2",
    "E",
    "ELSE_CORRELATED_ERROR",
    "CORRELATED_ERROR",
    "TICK_NOISE",  # just in case
}

def strip_noise_and_normalize_ticks(c: stim.Circuit) -> stim.Circuit:
    """Return a noise-free, repeat-expanded, normalized circuit."""
    # 1. Expand REPEAT k { ... }
    c = c.flattened()

    noise_gates = {
        "DEPOLARIZE1",
        "DEPOLARIZE2",
        "X_ERROR",
        "Y_ERROR",
        "Z_ERROR",
        "PAULI_CHANNEL_1",
        "PAULI_CHANNEL_2",
    }

    out = stim.Circuit()
    last_was_tick = False

    for inst in c:
        name = inst.name

        # Skip TICK if previous was TICK
        if name == "TICK":
            if last_was_tick:
                continue
            out.append("TICK")
            last_was_tick = True
            continue
        last_was_tick = False

        # Remove noise instructions
        if name in noise_gates:
            continue

        # Keep everything else (including DETECTOR / OBS / SHIFT_COORDS)
        out.append(name, inst.targets_copy(), inst.gate_args_copy())

    return out


# ----------------------------
# 3) Extract detector coords (respecting SHIFT_COORDS)
# ----------------------------

def detector_coords(c: stim.Circuit) -> List[Tuple[float, float, float]]:
    """Return detector coords in detector-index order, after applying SHIFT_COORDS."""
    sx = sy = st = 0.0
    coords: List[Tuple[float, float, float]] = []
    for inst in c:
        if inst.name == "SHIFT_COORDS":
            args = inst.gate_args_copy()
            dx = float(args[0]) if len(args) > 0 else 0.0
            dy = float(args[1]) if len(args) > 1 else 0.0
            dt = float(args[2]) if len(args) > 2 else 0.0
            sx += dx
            sy += dy
            st += dt
        elif inst.name == "DETECTOR":
            args = inst.gate_args_copy()
            x = float(args[0]) if len(args) > 0 else 0.0
            y = float(args[1]) if len(args) > 1 else 0.0
            t = float(args[2]) if len(args) > 2 else 0.0
            coords.append((x + sx, y + sy, t + st))
    return coords


# ----------------------------
# 4) DEM hyperedges (this is the “real” detector hypergraph)
# ----------------------------

@dataclass(frozen=True)
class Hyperedge:
    dets: Tuple[int, ...]
    obs: Tuple[int, ...]


def dem_hyperedges(c: stim.Circuit) -> Tuple[int, int, collections.Counter]:
    """
    Returns:
      (num_detectors, num_observables, multiset_of_hyperedges)
    where edges come from DEM "error" instructions after decomposition.
    """
    dem = c.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)

    edges = collections.Counter()
    for inst in dem:
        if inst.type != "error":
            continue
        dets: List[int] = []
        obs: List[int] = []
        for t in inst.targets_copy():
            if t.is_relative_detector_id():
                dets.append(t.val)
            elif t.is_logical_observable_id():
                obs.append(t.val)
        edges[Hyperedge(tuple(sorted(dets)), tuple(sorted(obs)))] += 1

    return dem.num_detectors, dem.num_observables, edges


# ----------------------------
# 5) Find detector-index mapping using coords (allow symmetry + translation)
# ----------------------------

def _xy_transforms():
    # 8 dihedral symmetries of the square on (x,y)
    #   (x,y), (x,-y), (-x,y), (-x,-y),
    #   (y,x), (y,-x), (-y,x), (-y,-x)
    def f1(x, y): return ( x,  y)
    def f2(x, y): return ( x, -y)
    def f3(x, y): return (-x,  y)
    def f4(x, y): return (-x, -y)
    def f5(x, y): return ( y,  x)
    def f6(x, y): return ( y, -x)
    def f7(x, y): return (-y,  x)
    def f8(x, y): return (-y, -x)
    return [f1,f2,f3,f4,f5,f6,f7,f8]


def find_detector_mapping_by_coords(
    ours_coords: List[Tuple[float,float,float]],
    stim_coords: List[Tuple[float,float,float]],
) -> Optional[Dict[int, int]]:
    """
    Attempts to map ours detector indices -> stim detector indices by matching (x,y,t),
    allowing dihedral symmetries in (x,y) and translation in (x,y) and in t.

    Assumes coords are (almost) unique. If duplicates exist, mapping may fail.
    """
    if len(ours_coords) != len(stim_coords):
        return None

    # Build reverse lookup for stim coords.
    stim_index_by_coord = {}
    for j, c in enumerate(stim_coords):
        if c in stim_index_by_coord:
            # duplicate coords -> ambiguous
            return None
        stim_index_by_coord[c] = j

    # Choose a few anchors from ours to try translations.
    anchors = list(range(min(5, len(ours_coords))))
    xy_fns = _xy_transforms()

    for f in xy_fns:
        for a in anchors:
            ox, oy, ot = ours_coords[a]
            tx, ty = f(ox, oy)

            # Try to align this to each of a few stim anchors (avoid O(n^2) brute force).
            stim_anchors = list(range(min(5, len(stim_coords))))
            for b in stim_anchors:
                sx, sy, st = stim_coords[b]
                dx = sx - tx
                dy = sy - ty
                dt = st - ot

                mapping: Dict[int, int] = {}
                ok = True
                for i, (x, y, t) in enumerate(ours_coords):
                    x2, y2 = f(x, y)
                    c2 = (x2 + dx, y2 + dy, t + dt)
                    j = stim_index_by_coord.get(c2)
                    if j is None:
                        ok = False
                        break
                    mapping[i] = j

                if ok and len(mapping) == len(ours_coords):
                    return mapping

    return None


def remap_edges(
    edges: collections.Counter,
    mapping: Dict[int, int],
) -> collections.Counter:
    out = collections.Counter()
    for e, count in edges.items():
        dets2 = tuple(sorted(mapping[d] for d in e.dets))
        out[Hyperedge(dets2, e.obs)] += count
    return out


# ----------------------------
# 6) Main
# ----------------------------

def main(d=5, rounds=4, basis="Z"):
    ours_raw = build_ours(d, rounds, basis)
    stim_raw = build_stim(d, rounds, basis)

    ours = strip_noise_and_normalize_ticks(ours_raw)
    ours.diagram(
        'detslice-with-ops-svg',
        tick=range(8, 18),
        filter_coords=['D81'],
    )

    stimc = strip_noise_and_normalize_ticks(stim_raw)

    print(f"Comparing rotated surface code circuits for d={d}, rounds={rounds}, basis={basis}")
    print(f"  ours:  qubits={ours.num_qubits} {str(ours_raw)}")
    print(f"  stim:  qubits={stimc.num_qubits} {str(stim_raw)}")

    ours_dcoords = detector_coords(ours)
    stim_dcoords = detector_coords(stimc)
    print(f"  ours:  dets={len(ours_dcoords)}")
    print(f"  stim:  dets={len(stim_dcoords)}")

    # Build DEM hyperedges
    try:
        o_nd, o_no, o_edges = dem_hyperedges(ours)
    except ValueError as ex:
        print("\n[OURS] detector_error_model failed:")
        print(ex)
        return

    try:
        s_nd, s_no, s_edges = dem_hyperedges(stimc)
    except ValueError as ex:
        print("\n[STIM] detector_error_model failed:")
        print(ex)
        return

    if (o_nd, o_no) != (s_nd, s_no):
        print(f"\nDifferent DEM sizes: ours (dets={o_nd}, obs={o_no}) vs stim (dets={s_nd}, obs={s_no})")
        return

    mapping = find_detector_mapping_by_coords(ours_dcoords, stim_dcoords)
    if mapping is None:
        print("\nCould not find a detector mapping by coords (maybe coord mismatch or duplicate coords).")
        return

    o_edges_remapped = remap_edges(o_edges, mapping)
    if o_edges_remapped == s_edges:
        print("\n✅ DEM hypergraphs match (isomorphic via detector coord mapping).")
    else:
        print("\n❌ DEM hypergraphs differ.")
        # Give a tiny diff.
        only_in_ours = o_edges_remapped - s_edges
        only_in_stim = s_edges - o_edges_remapped
        print(f"  edges only in ours: {sum(only_in_ours.values())}")
        print(f"  edges only in stim: {sum(only_in_stim.values())}")
        # Print a few examples
        for k in list(only_in_ours.keys())[:5]:
            print("  ours-only example:", k, "x", only_in_ours[k])
        for k in list(only_in_stim.keys())[:5]:
            print("  stim-only example:", k, "x", only_in_stim[k])

    
    

if __name__ == "__main__":
    main(d=5, rounds=4, basis="Z")

Comparing rotated surface code circuits for d=5, rounds=4, basis=Z
  ours:  qubits=49 QUBIT_COORDS(1, 1) 0
QUBIT_COORDS(3, 1) 1
QUBIT_COORDS(5, 1) 2
QUBIT_COORDS(7, 1) 3
QUBIT_COORDS(9, 1) 4
QUBIT_COORDS(1, 3) 5
QUBIT_COORDS(3, 3) 6
QUBIT_COORDS(5, 3) 7
QUBIT_COORDS(7, 3) 8
QUBIT_COORDS(9, 3) 9
QUBIT_COORDS(1, 5) 10
QUBIT_COORDS(3, 5) 11
QUBIT_COORDS(5, 5) 12
QUBIT_COORDS(7, 5) 13
QUBIT_COORDS(9, 5) 14
QUBIT_COORDS(1, 7) 15
QUBIT_COORDS(3, 7) 16
QUBIT_COORDS(5, 7) 17
QUBIT_COORDS(7, 7) 18
QUBIT_COORDS(9, 7) 19
QUBIT_COORDS(1, 9) 20
QUBIT_COORDS(3, 9) 21
QUBIT_COORDS(5, 9) 22
QUBIT_COORDS(7, 9) 23
QUBIT_COORDS(9, 9) 24
QUBIT_COORDS(2, 0) 25
QUBIT_COORDS(2, 4) 26
QUBIT_COORDS(2, 8) 27
QUBIT_COORDS(4, 2) 28
QUBIT_COORDS(4, 6) 29
QUBIT_COORDS(4, 10) 30
QUBIT_COORDS(6, 0) 31
QUBIT_COORDS(6, 4) 32
QUBIT_COORDS(6, 8) 33
QUBIT_COORDS(8, 2) 34
QUBIT_COORDS(8, 6) 35
QUBIT_COORDS(8, 10) 36
QUBIT_COORDS(0, 4) 37
QUBIT_COORDS(0, 8) 38
QUBIT_COORDS(2, 2) 39
QUBIT_COORDS(2, 6) 40
QUBIT_COORDS(4, 4) 4

In [4]:
# 1. Tiny run to inspect.
code = RotatedSurfaceCode(3)
noise = CircuitDepolarizingNoise(p1=0.001, p2=0.001)
exp = CSSMemoryExperiment(code=code, noise_model=noise, rounds=3, basis="Z")

c = exp.to_stim()
noisy = noise.apply(c)
dem = noisy.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
print(dem)
print("detectors:", dem.num_detectors, "observables:", dem.num_observables)

sampler = dem.compile_sampler()
det, obs, _ = sampler.sample(shots=20)
print("det shape:", det.shape, "obs shape:", obs.shape)

from qectostim.decoders.pymatching_decoder import PyMatchingDecoder
decoder = PyMatchingDecoder(dem=dem)
corr = decoder.decode_batch(det)
print("corr shape:", corr.shape)
print("first det row:", det[0])
print("first corr row:", corr[0])

error(0.001864356503703524) D0 D8
error(0.000533333333333148) D0 D8 ^ D5 L0
error(0.002395701190099389) D1 D9
error(0.0002667378157289138) D1 D9 ^ D5 D12
error(0.0002667378157289138) D1 D9 ^ D6
error(0.0002667378157289138) D1 D9 ^ D12
error(0.0002667378157289138) D1 D9 ^ D13 D14
error(0.002395701190099389) D2 D10
error(0.0002667378157289138) D2 D10 ^ D6 D7
error(0.0002667378157289138) D2 D10 ^ D6 D13
error(0.0002667378157289138) D2 D10 ^ D13 L0
error(0.0002667378157289138) D2 D10 ^ D15 L0
error(0.001864356503703524) D3 D11
error(0.0002667378157289138) D3 D11 ^ D6
error(0.0002667378157289138) D3 D11 ^ D14
error(0.0002667378157289138) D4
error(0.0002667378157289138) D4 D5
error(0.0002667378157289138) D4 D12
error(0.0002667378157289138) D5 D6
error(0.0007997866287252842) D5 D12
error(0.000533333333333148) D5 D13
error(0.001066097777777407) D5 L0
error(0.001066097777777407) D6
error(0.000533333333333148) D6 D7
error(0.0007997866287252842) D6 D13
error(0.000533333333333148) D6 D14
error(0.0

In [5]:
import stim
from qectostim.codes.base.rotated_surface import RotatedSurfaceCode
from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise

code = RotatedSurfaceCode(5)
noise = CircuitDepolarizingNoise(p1=1e-3, p2=1e-3)
exp = CSSMemoryExperiment(code=code, noise_model=noise, rounds=1, basis="Z")

noisy = noise.apply(exp.to_stim())
dem = noisy.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)

print(dem)
print("num_detectors:", dem.num_detectors)
print("num_observables:", dem.num_observables)

sampler = dem.compile_sampler()
det, obs, _ = sampler.sample(shots=20)
print("det shape:", det.shape)
print("obs shape:", obs.shape)
print("det:\n", det.astype(int))
print("obs:\n", obs.astype(int))

error(0.0002667378157289138) D0 D2
error(0.0002667378157289138) D0 D3
error(0.000533333333333148) D0 D12
error(0.0002667378157289138) D1
error(0.0002667378157289138) D1 D3
error(0.0002667378157289138) D1 D13
error(0.0002667378157289138) D2 D4
error(0.001066097777777407) D2 D12
error(0.000533333333333148) D2 D14
error(0.001598293940147593) D2 L0
error(0.0007997866287252842) D3 D4
error(0.0002667378157289138) D3 D5
error(0.001066097777777407) D3 D12
error(0.001066097777777407) D3 D13
error(0.001066097777777407) D3 D15
error(0.0007997866287252842) D4 D6
error(0.0002667378157289138) D4 D7
error(0.001066097777777407) D4 D14
error(0.001066097777777407) D4 D15
error(0.001066097777777407) D4 D16
error(0.001598293940147593) D5
error(0.0007997866287252842) D5 D7
error(0.001066097777777407) D5 D15
error(0.000533333333333148) D5 D17
error(0.0002667378157289138) D6 D8
error(0.001066097777777407) D6 D16
error(0.000533333333333148) D6 D18
error(0.001598293940147593) D6 L0
error(0.0007997866287252842)

In [6]:
import stim

circ = stim.Circuit.generated(
    "surface_code:rotated_memory_z",
    distance=5,
    rounds=5,
    after_clifford_depolarization=1e-3,
    before_measure_flip_probability=1e-3,
    before_round_data_depolarization=1e-3,
)
print('stims circuit')
print(circ)

code = RotatedSurfaceCode(5)
noise = CircuitDepolarizingNoise(p1=1e-3, p2=1e-3)
exp = CSSMemoryExperiment(code=code, noise_model=noise, rounds=5, basis="Z")

ideal = exp.to_stim()
noisy = noise.apply(ideal)

print('our circuit')
print(noisy)
# res_no_dec = exp.run_no_decode(shots=200_000)
# res_dec    = exp.run_decode  (shots=200_000)
# print("no decode:", res_no_dec["logical_error_rate"])
# print("decode   :", res_dec["logical_error_rate"])

stims circuit
QUBIT_COORDS(1, 1) 1
QUBIT_COORDS(2, 0) 2
QUBIT_COORDS(3, 1) 3
QUBIT_COORDS(5, 1) 5
QUBIT_COORDS(6, 0) 6
QUBIT_COORDS(7, 1) 7
QUBIT_COORDS(9, 1) 9
QUBIT_COORDS(1, 3) 12
QUBIT_COORDS(2, 2) 13
QUBIT_COORDS(3, 3) 14
QUBIT_COORDS(4, 2) 15
QUBIT_COORDS(5, 3) 16
QUBIT_COORDS(6, 2) 17
QUBIT_COORDS(7, 3) 18
QUBIT_COORDS(8, 2) 19
QUBIT_COORDS(9, 3) 20
QUBIT_COORDS(10, 2) 21
QUBIT_COORDS(0, 4) 22
QUBIT_COORDS(1, 5) 23
QUBIT_COORDS(2, 4) 24
QUBIT_COORDS(3, 5) 25
QUBIT_COORDS(4, 4) 26
QUBIT_COORDS(5, 5) 27
QUBIT_COORDS(6, 4) 28
QUBIT_COORDS(7, 5) 29
QUBIT_COORDS(8, 4) 30
QUBIT_COORDS(9, 5) 31
QUBIT_COORDS(1, 7) 34
QUBIT_COORDS(2, 6) 35
QUBIT_COORDS(3, 7) 36
QUBIT_COORDS(4, 6) 37
QUBIT_COORDS(5, 7) 38
QUBIT_COORDS(6, 6) 39
QUBIT_COORDS(7, 7) 40
QUBIT_COORDS(8, 6) 41
QUBIT_COORDS(9, 7) 42
QUBIT_COORDS(10, 6) 43
QUBIT_COORDS(0, 8) 44
QUBIT_COORDS(1, 9) 45
QUBIT_COORDS(2, 8) 46
QUBIT_COORDS(3, 9) 47
QUBIT_COORDS(4, 8) 48
QUBIT_COORDS(5, 9) 49
QUBIT_COORDS(6, 8) 50
QUBIT_COORDS(7, 9) 51
Q

In [13]:
import stim
import numpy as np

from qectostim.codes.base.rotated_surface import RotatedSurfaceCode
from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.noise.models import NoiseModel  # if needed; we will set noise=None

# ---------------------------------------------------------
# Helpers
# ---------------------------------------------------------

def strip_noise_and_normalize(c: stim.Circuit) -> stim.Circuit:
    """Remove noise channels, normalize TICK placement, and drop QUBIT_COORDS."""
    clean = stim.Circuit()

    for inst in c:
        name = inst.name

        # Remove noise channels
        if name in [
            "DEPOLARIZE1", "DEPOLARIZE2",
            "X_ERROR", "Y_ERROR", "Z_ERROR",
            "PAULI_CHANNEL_1", "PAULI_CHANNEL_2"
        ]:
            continue

        # Keep everything else
        if name == "QUBIT_COORDS":
            # Coordinates differ; omit for structural comparison
            continue

        clean.append(inst)

    # Normalize TICK spacing by collapsing consecutive TICKs
    normalized = stim.Circuit()
    last_was_tick = False
    for inst in clean:
        if inst.name == "TICK":
            if last_was_tick:
                continue
            last_was_tick = True
        else:
            last_was_tick = False
        normalized.append(inst)

    return normalized


def detector_hypergraph(c: stim.Circuit) -> stim.DetectorErrorModel:
    """Return DEM with decomposed errors (stable structural representation)."""
    return c.detector_error_model(decompose_errors=True)


def compare_dems(d1: stim.DetectorErrorModel, d2: stim.DetectorErrorModel):
    """Check if two DEMs are isomorphic (structurally identical)."""
    # For now: direct string equality after sorting error mechanisms
    # (this is a good practical proxy for DEM isomorphism).
    s1 = str(d1)
    s2 = str(d2)

    if s1 == s2:
        print("\n✅ DEMs match exactly — circuits are equivalent.")
        return True

    print("\n❌ DEMs differ.")
    print("---- DEM 1 ----")
    print(s1)
    print("\n---- DEM 2 ----")
    print(s2)

    return False


# ---------------------------------------------------------
# Build OUR circuit
# ---------------------------------------------------------

def build_ours(d=5, rounds=4):
    code = RotatedSurfaceCode(d)
    exp = CSSMemoryExperiment(code, rounds=rounds, noise_model=None, basis="Z")
    circ = exp.to_stim()
    return circ


# ---------------------------------------------------------
# Build STIM'S circuit
# ---------------------------------------------------------

def build_stim(d=5, rounds=4):
    return stim.Circuit.generated(
        "surface_code:rotated_memory_z",
        rounds=rounds,
        distance=d,

    )


# ---------------------------------------------------------
# Main test
# ---------------------------------------------------------

if __name__ == "__main__":
    d = 5
    rounds = 4
    print(f"Comparing rotated surface code circuits for d={d}, rounds={rounds}")

    ours_raw = build_ours(d, rounds)
    stim_raw = build_stim(d, rounds)

    ours = strip_noise_and_normalize(ours_raw)
    stimc = strip_noise_and_normalize(stim_raw)

    print("\n=== Normalized Our Circuit ===")
    print(ours[:300])  # first 300 lines

    print("\n=== Normalized Stim Circuit ===")
    print(stimc[:300])

    # Build DEMs
    dem_ours = detector_hypergraph(ours)
    dem_stim = detector_hypergraph(stimc)

    # Compare
    compare_dems(dem_ours, dem_stim)

    print("\nDone.")

Comparing rotated surface code circuits for d=5, rounds=4

=== Normalized Our Circuit ===
R 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
TICK
H 25 26 27 28 29 30 31 32 33 34 35 36
TICK
CX 1 25 11 26 21 27 7 28 17 29 3 31 13 32 23 33 9 34 19 35 10 37 20 38 6 39 16 40 12 41 22 42 8 43 18 44 14 45 24 46
TICK
CX 0 25 10 26 20 27 6 28 16 29 2 31 12 32 22 33 8 34 18 35 5 37 15 38 1 39 11 40 7 41 17 42 3 43 13 44 9 45 19 46
TICK
CX 5 26 15 27 1 28 11 29 21 30 7 32 17 33 3 34 13 35 23 36 0 39 10 40 6 41 16 42 2 43 12 44 8 45 18 46 4 47 14 48
TICK
CX 6 26 16 27 2 28 12 29 22 30 8 32 18 33 4 34 14 35 24 36 5 39 15 40 11 41 21 42 7 43 17 44 13 45 23 46 9 47 19 48
H 25 26 27 28 29 30 31 32 33 34 35 36
MR 25 26 27 28 29 30 31 32 33 34 35 36
DETECTOR(2, 0, 0) rec[-12]
DETECTOR(2, 4, 0) rec[-11]
DETECTOR(2, 8, 0) rec[-10]
DETECTOR(4, 2, 0) rec[-9]
DETECTOR(4, 6, 0) rec[-8]
DETECTOR(4, 10, 0) rec[-7]
DETECTOR(6

In [9]:
for d in [3,5,7]:

    code = RotatedSurfaceCode(d)
    noise = CircuitDepolarizingNoise(p1=1e-6, p2=1e-6)
    exp = CSSMemoryExperiment(code=code, noise_model=noise, rounds=3, basis="Z")

    ideal = exp.to_stim()
    noisy = noise.apply(ideal)

    # print("Ideal length:", len(str(ideal).splitlines()))
    # print("Noisy length:", len(str(noisy).splitlines()))
    # print("Has DEPOLARIZE?", "DEPOLARIZE" in str(noisy))

    # dem = noisy.detector_error_model(decompose_errors=True)
    # print("DEM summary:", dem)
    res = exp.run_decode(1_000_000)

    print(f"CODE DISTANCE {d} with {res}")

[run_decode] --- starting ---
[run_decode] shots           = 1000000
[run_decode] decoder_name    = None
[run_decode] code type       = <class 'qectostim.codes.base.rotated_surface.RotatedSurfaceCode'>
[run_decode] noise_model     = <class 'qectostim.noise.models.CircuitDepolarizingNoise'>
[run_decode] base circuit    = 83 instructions
[run_decode] noisy circuit   = 100 instructions
[run_decode] DEM: detectors   = 24
[run_decode] DEM: errors      = 91
[run_decode] DEM: observables = 1
[run_decode] DEM snippet:
 error(1.866664355573657e-06) D0 D8
error(5.333333332976022e-07) D0 D8 ^ D5 L0
error(2.399995697762614e-06) D1 D9
error(2.666667377599501e-07) D1 D9 ^ D5 D12
error(2.666667377599501e-07) D1 D9 ^ D6
error(2.666667377599501e-07) D1 D9 ^ D12
error(2.666667377599501e-07) D1 D9 ^ D13 D14
error(2.399995697
[run_decode] decoder type    = <class 'qectostim.decoders.pymatching_decoder.PyMatchingDecoder'>
[run_decode] sampling DEM directly...
[run_decode] type(raw)       = <class 'tuple'>


In [10]:
from qectostim.codes.base.four_two_two import FourQubit422Code
from qectostim.codes.base.six_two_two import SixQubit622Code
from qectostim.codes.composite.concatenated import ConcatenatedTopologicalCSSCode
from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise

noise = CircuitDepolarizingNoise(p1=1e-3, p2=1e-3)

outer = SixQubit622Code()
inner = FourQubit422Code()
concat = ConcatenatedTopologicalCSSCode(outer, inner)

exp = CSSMemoryExperiment(code=concat, noise_model=noise, rounds=3)
result = exp.run_decode(shots=1000)
result

[run_decode] --- starting ---
[run_decode] shots           = 1000
[run_decode] decoder_name    = None
[run_decode] code type       = <class 'qectostim.codes.composite.concatenated.ConcatenatedTopologicalCSSCode'>
[run_decode] noise_model     = <class 'qectostim.noise.models.CircuitDepolarizingNoise'>
[run_decode] base circuit    = 135 instructions
[run_decode] noisy circuit   = 200 instructions
[run_decode] DEM: detectors   = 48
[run_decode] DEM: errors      = 78
[run_decode] DEM: observables = 1
[run_decode] DEM snippet:
 error(0.002395701190099389) D0 D18
error(0.001066097777777407) D0 D18 ^ D12 L0
error(0.002395701190099389) D1 D19
error(0.001066097777777407) D1 D19 ^ D12 L0
error(0.002395701190099389) D2 D20
error(0.001066097777777407) D2 D20 ^ D13 L0
error(0.002395701190099389) D3 D21
error(0.001066097777777407) 
[run_decode] decoder type    = <class 'qectostim.decoders.pymatching_decoder.PyMatchingDecoder'>
[run_decode] sampling DEM directly...
[run_decode] type(raw)       = <cla

{'shots': 1000,
 'logical_errors': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    